In [6]:
import pandas as pd
import numpy as np
import scipy

prec = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/Lebanon_to_StVincent_pr_1991_2015.xls')
temp = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/Lebanon_to_StVincent_temp_1991_2015.xls')
occur = pd.read_excel('/Users/RodolfoSaldanha/Desktop/IP/final_project/Datasets/aegypti_albopictus.xlsx')

In [7]:
temp.rename(columns={'\tYear':'Year', ' Month':'Month', ' Country':'Country'}, inplace=True)
prec.rename(columns={'\tYear':'Year', ' Month':'Month', ' Country':'Country'}, inplace=True)
occur.rename(columns={'YEAR':'Year', 'COUNTRY_ID':'Country'}, inplace=True)

In [8]:
# Drop unnecessary columns
occur.drop(columns=['SOURCE_TYPE','LOCATION_TYPE','POLYGON_ADMIN','STATUS','GAUL_AD0','X','Y'],inplace=True)
occur.head()

,VECTOR,OCCURRENCE_ID,Year,COUNTRY,Country
0,Aedes aegypti,1,1958,Kenya,KEN
1,Aedes aegypti,2,1960,Congo,COG
2,Aedes aegypti,3,1960,Congo,COG
3,Aedes aegypti,4,1960,Kenya,KEN
4,Aedes aegypti,5,1960,Kenya,KEN


In [9]:
occur['Year'] = pd.to_numeric(occur['Year'], downcast='integer',errors='coerce')
occur.describe()

,OCCURRENCE_ID,Year
count,42066.000000,40804.000000
mean,21033.500000,2006.723630
std,12143.552549,7.616601
min,1.000000,1958.000000
25%,10517.250000,2004.000000
50%,21033.500000,2008.000000
75%,31549.750000,2013.000000
max,42066.000000,2014.000000


In [10]:
# Select the timeframe of temperature and rainfall we colected 
occur = occur.loc[(occur['Year']>=1991) & (occur['Year']<=2014)]

# Different abbreviation of the same country in different datasets
occur['Country'] = occur['Country'].replace('ROU', 'ROM')

# Set as NaN the countries we didn't find the temperature nor the rainfall
occur['Country'] = occur['Country'].replace('XXX', np.nan)
occur['Country'] = occur['Country'].replace('TWN', np.nan)
occur['Country'] = occur['Country'].replace('HKG', np.nan)

occur['Year'] = occur['Year'].astype(int)
occur.dropna(inplace=True)
occur.dtypes

VECTOR           object
OCCURRENCE_ID     int64
Year              int64
COUNTRY          object
Country          object
dtype: object

In [11]:
annualOccur = pd.DataFrame(occur.groupby(['Year','Country'])['OCCURRENCE_ID'].count()).reset_index()
annualOccur

,Year,Country,OCCURRENCE_ID
0,1991,AUS,31
1,1991,BRA,2
2,1991,COL,1
3,1991,FJI,4
4,1991,GUF,1
5,1991,IND,1
6,1991,ITA,17
7,1991,MAC,1
8,1991,MEX,1
9,1991,MYS,1


In [24]:
annualTemp = pd.DataFrame(temp.groupby(['Year','Country'])['tas'].sum()/12).reset_index()
annualPrec = pd.DataFrame(prec.groupby(['Year','Country'])['pr'].sum()/12).reset_index()
temp_rain=pd.merge(annualTemp,annualPrec,
                   how='inner',
                   left_on = ['Year','Country'],
                   right_on = ['Year','Country'])

In [25]:
temperature=temp.drop_duplicates()
temperature.to_csv('temperature.csv', encoding='utf-8', index=False)
temperature.dtypes

tas        float64
Year         int64
Month        int64
Country     object
 ISO3      float64
 ISO2      float64
dtype: object

In [26]:
precipitation=prec.drop_duplicates()
precipitation.to_csv('precipitation.csv', encoding='utf-8', index=False)
precipitation.describe()

,pr,Year,Month,ISO3,ISO2
count,44400.000000,44400.000000,44400.000000,0.0,0.0
mean,135.276355,2003.000000,6.500000,NaN,NaN
std,133.079973,7.211184,3.452091,NaN,NaN
min,0.000000,1991.000000,1.000000,NaN,NaN
25%,42.175175,1997.000000,3.750000,NaN,NaN
50%,101.266500,2003.000000,6.500000,NaN,NaN
75%,194.548750,2009.000000,9.250000,NaN,NaN
max,2882.040000,2015.000000,12.000000,NaN,NaN


In [27]:
temp_rain.to_csv('temp_rain.csv', encoding='utf-8', index=False)

In [42]:
temp_rain.head()

,Year,Country,tas,pr
0,1991,ABW,28.078008,38.882968
1,1991,AFG,13.118033,36.931453
2,1991,AGO,21.652200,83.844948
3,1991,AIA,26.108292,113.631158
4,1991,ALB,11.038935,77.587033


In [41]:
tempByYear = pd.pivot_table(temp_rain,
                        values="tas",
                        index=['Country'],
                        columns=['Year'])
tempByYear.isnull().sum()

Year
1991    0
1992    0
1993    0
1994    0
1995    0
1996    0
1997    0
1998    0
1999    0
2000    0
2001    0
2002    0
2003    0
2004    0
2005    0
2006    0
2007    0
2008    0
2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
dtype: int64

In [40]:
rainByYear = pd.pivot_table(temp_rain,
                        values="pr",
                        index=['Country'],
                        columns=['Year'])
rainByYear.isnull().sum()

Year
1991    0
1992    0
1993    0
1994    0
1995    0
1996    0
1997    0
1998    0
1999    0
2000    0
2001    0
2002    0
2003    0
2004    0
2005    0
2006    0
2007    0
2008    0
2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
dtype: int64

In [43]:
temp_rain_occur=pd.merge(annualOccur,temp_rain,
                   how='left',
                   left_on = ['Year','Country'],
                   right_on = ['Year','Country'])
temp_rain_occur.rename(columns={'OCCURRENCE_ID':'Ocurrences', 'tas':'Temperature', 'pr':'Precipitation'}, inplace=True)
temp_rain_occur

,Year,Country,Ocurrences,Temperature,Precipitation
0,1991,AUS,31,22.172075,37.912953
1,1991,BRA,2,25.100117,139.381483
2,1991,COL,1,24.545633,173.873683
3,1991,FJI,4,24.151858,223.556367
4,1991,GUF,1,25.670833,218.727517
5,1991,IND,1,24.220583,83.869383
6,1991,ITA,17,11.681567,74.369725
7,1991,MAC,1,23.240850,139.311213
8,1991,MEX,1,21.018242,63.625225
9,1991,MYS,1,25.663675,228.281667


In [44]:
temp_rain_occur.to_csv('ocurences.csv', encoding='utf-8', index=False)